In [50]:
import numpy as np
import pickle

In [53]:
file_name ='all tasks, ego V density 0.03, time length 1000, mode pso.pkl'
with open(file_name,'rb') as file:
    logs  = pickle.loads(file.read())
print(len(logs))

27


In [54]:
file_name ='finished tasks, ego V density 0.03, time length 1000, mode pso.pkl'
with open(file_name,'rb') as file:
    finsih_logs  = pickle.loads(file.read())
print(len(finsih_logs))

63


In [52]:
print(len(logs))

27


In [55]:
max_time = 0
for log in finsih_logs:
    max_time = max(max_time,log.total_time)
    print(log.total_time)
print('max time', max_time)

105
105
140
118
192
264
196
80
222
293
74
73
262
370
279
404
337
420
454
354
208
463
485
249
501
452
218
238
592
301
409
655
595
494
752
200
716
523
770
189
270
254
157
343
307
330
333
79
190
190
302
64
525
259
165
526
513
195
148
172
172
323
197
max time 770


In [56]:
vars(log)

{'tid': 26,
 'vid': 14,
 'start_time': 962,
 'end_time': 1156,
 'total_time': 197,
 'time_vehicle_compute': 76,
 'time_ul_transmit': 16,
 'time_dl_transmit': 105,
 'remain_compute_size': 0.0,
 'task_size': 79,
 'remain_size_dl': 0.0,
 'result_size': 24,
 'remain_size_ul': 0.0,
 'generated_vid': 0,
 'experienced_resource_allocation': 10.0}

In [57]:
max_left_time = 0
for log in finsih_logs:
    time_left = log.total_time -log.time_dl_transmit
    max_left_time = max(max_left_time,time_left)
    print(time_left)
print('max_left_time',max_left_time)

44
44
80
28
99
62
99
59
89
110
52
63
91
84
104
94
107
115
168
162
124
101
134
164
172
138
149
198
252
206
222
309
254
260
305
178
305
305
353
140
233
205
140
306
257
305
305
76
158
158
273
58
468
230
148
479
467
178
110
98
98
281
92
max_left_time 479


In [31]:
print(log)

In [35]:
for log in logs:
    print(log.end_time)

1148
2046
2747
4758
5392
7399
8409
8970
9085
9714
9753
9956
10375
12261
12348
12452
12891
12906
14051
15481
15952
16132
16683
16741
16802
17034
17254
18298
18941
19037
19165
19676
19734
19735
20569
20671
20764
20766
22685
23441
23606
25076
25643
27128
27193
27656
28356
28371
30682
31388
31448
31544
31816
32144
32641
32786
33496
33647
33796
34509
35330
36365
36912
37094
37555
37882
39029
39558
39650
40077
40139
40330
40374
40394
40706
40750
41237
41607
41761
43697
44160
44223
45066
45126
45520
45762
46585
46675
46750
47368
47472
47911
49046
49204
49710
49791
50285
51182
53474
54646
54814
55521
56043
56711
56788
56898
57037
57100
57192
57773
59148
59601
59761
